In [41]:
!uv pip install -q --system numba-cuda==0.4.0
from numba import config
config.CUDA_ENABLE_PYNVJITLINK = 1
config.CUDA_LOW_OCCUPANCY_WARNINGS = 0


In [9]:
# implementando multiplicação de matrizes com paralismo

from numba import cuda
import numpy as np
import math
import time
import torch

In [54]:
m1, m2 = np.random.randn(12800, 128), np.random.randn(128, 1280)

In [48]:
# matmul bruto

%%timeit -n 1 -r 1
out = np.zeros((m1.shape[0],m2.shape[1]))

for i in range(m1.shape[0]):
  for j in range(m2.shape[1]):
    for k in range(m1.shape[1]):
      out[i,j] += m1[i,k]*m2[k,j]


KeyboardInterrupt: 

In [55]:
# matmul elegante
%%timeit -n 1 -r 1
m1_tensor = torch.tensor(m1)
m2_tensor = torch.tensor(m2)
out = torch.zeros((m1.shape[0],m2.shape[1]))
for i in range(m1.shape[0]):
  out[i] = (m1_tensor[i,:,None] * m2_tensor).sum(dim=0)

2.69 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [56]:
@cuda.jit
def mat_mul(m1, m2, out):
    # Corrected typo and summation logic
    i, j = cuda.grid(2)

    if i < out.shape[0] and j < out.shape[1]:
        tmp = 0.
        for k in range(m1.shape[1]):
            tmp += m1[i, k] * m2[k, j]
        out[i, j] = tmp

In [67]:
TPB = 64

# Corrected to use shape attributes
BPG = math.ceil(m1.shape[0] / TPB), math.ceil(m2.shape[1] / TPB)

In [68]:
%%timeit -n 1 -r 1
m1_tensor = cuda.to_device(m1)
m2_tensor = cuda.to_device(m2)
out = np.zeros((m1.shape[0],m2.shape[1]))
out = cuda.to_device(out)
mat_mul[BPG,TPB](m1,m2,out)
t = out.copy_to_host()

#potencia


107 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


/usr/local/lib/python3.11/dist-packages/numba_cuda/numba/cuda/cudadrv/devicearray.py:890: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  ):


In [72]:
%%timeit -n 1 -r 1

m1_t = torch.tensor(m1).cuda()
m2_t = torch.tensor(m2).cuda()
out = (m1_t@m2_t).cpu()

#receba o nosso é melhor

162 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
